## 1. Imports


In [ ]:
import os
import sys
import gc
import warnings

import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torchvision.transforms import Compose, ToTensor, Resize, Normalize, Lambda
from PIL import PngImagePlugin

from IPython.display import clear_output

sys.path.append("..")
from src.enot import SDE
from src.cunet import CUNet
from src.mnistm_utils import MNISTM
from src.fid_score import calculate_frechet_distance
from src.tools import (
    set_random_seed,
    get_loader_stats,
    get_sde_pushed_loader_stats,
    get_sde_pushed_loader_accuracy,
    get_sde_pushed_loader_metrics,
)
from src.plotters import (
    plot_sde_pushed_images,
    plot_sde_pushed_random_class_images,
)
from src.samplers import (
    SubsetGuidedSampler,
    SubsetGuidedDataset,
    get_indicies_subset,
)

LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

warnings.filterwarnings("ignore")

%matplotlib inline 

In [ ]:
gc.collect()
torch.cuda.empty_cache()

## 2. Init Config

the config file `config.json` is saved at saved_models/EXP_NAME/


In [ ]:
SEED = 0x3060
set_random_seed(SEED)

# dataset choosing
DATASET, DATASET_PATH = "fmnist2mnist", "../datasets/"
# DATASET, DATASET_PATH = "mnist2mnistm", "../datasets/"
# DATASET, DATASET_PATH = "mnist2usps", "../datasets/"
# DATASET, DATASET_PATH = "mnist2kmnist", "../datasets/"

IMG_SIZE = 32
DATASET1_CHANNELS = 1
DATASET2_CHANNELS = 1

# GPU choosing
DEVICE_IDS = [0]
assert torch.cuda.is_available()

CONTINUE = 0

# All hyperparameters below is set to the values used for the experiments, which discribed in the article

# training algorithm settings
BATCH_SIZE = 32
SUBSET_SIZE = 2  # 2 for testing
NUM_LABELED = 10  # num of labeled target in training set

T_ITERS = 10
MAX_STEPS = 2500 + 1  # 2501 for testing
INTEGRAL_SCALE = 1 / (3 * IMG_SIZE * IMG_SIZE)
EPSILON_SCHEDULER_LAST_ITER = 20000

# optimizer settings
D_LR, T_LR = 1e-4, 1e-4
BETA_D, BETA_T = 0.9, 0.9
T_GRADIENT_MAX_NORM = float(500)
D_GRADIENT_MAX_NORM = float(500)

# SDE network settings
EPSILON = 0  # [0 , 1, 10]
IMAGE_INPUT = True
PREDICT_SHIFT = True
N_STEPS = 5  #
UNET_BASE_FACTOR = 128
TIME_DIM = 128
USE_POSITIONAL_ENCODING = True
ONE_STEP_INIT_ITERS = 0
USE_GRADIENT_CHECKPOINT = False
N_LAST_STEPS_WITHOUT_NOISE = 1

# plot settings
GRAY_PLOTS = True
STEPS_TO_SHOW = 10

# log settings
SMART_INTERVALS = False
INTERVAL_SHRINK_START_TIME = 0.98
TRACK_VAR_INTERVAL = 100
PLOT_INTERVAL = 500
CPKT_INTERVAL = 500

FID_EPOCHS = 1

EXP_NAME = f"ENOT_Class_{DATASET}_{SEED}"
LOAD_PATH = f"../saved_models/{EXP_NAME}/"

if not os.path.exists(LOAD_PATH):
    raise "no such file or directory"

In [ ]:
source_subset = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
new_labels_source = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}
target_subset = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
new_labels_target = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [ ]:
classifier = torchvision.models.resnet18()
classifier.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)

source_transform = Compose(
    [
        Resize((IMG_SIZE, IMG_SIZE)),
        ToTensor(),
        Normalize((0.5), (0.5)),
    ]
)
target_transform = source_transform

if DATASET == "mnist2kmnist":
    source = torchvision.datasets.MNIST
    target = torchvision.datasets.KMNIST
    classifier.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    classifier.load_state_dict(torch.load("../saved_models/classifiers/kmnist.pt"))

elif DATASET == "fmnist2mnist":
    source = torchvision.datasets.FashionMNIST
    target = torchvision.datasets.MNIST
    classifier.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    classifier.load_state_dict(torch.load("../saved_models/classifiers/mnist.pt"))

elif DATASET == "mnist2usps":
    source = torchvision.datasets.MNIST
    target = torchvision.datasets.USPS
    classifier.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    classifier.load_state_dict(torch.load("../saved_models/classifiers/usps.pt"))

elif DATASET == "mnist2mnistm":
    DATASET1_CHANNELS = 3
    DATASET2_CHANNELS = 3
    classifier.load_state_dict(torch.load("../saved_models/classifiers/mnistm.pt"))
    GRAY_PLOTS = False
    source = torchvision.datasets.MNIST
    target = MNISTM
    source_transform = Compose(
        [
            Resize((IMG_SIZE, IMG_SIZE)),
            ToTensor(),
            Normalize((0.5), (0.5)),
            Lambda(lambda x: -x.repeat(3, 1, 1)),
        ]
    )
    target_transform = Compose(
        [Resize(IMG_SIZE), ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

classifier.cuda()
classifier.eval()

## 3. Initialize samplers


In [ ]:
source_test = source(
    root=DATASET_PATH, train=False, download=True, transform=source_transform
)
source_subset_samples, source_labels, source_class_indicies = get_indicies_subset(
    source_test,
    new_labels=new_labels_source,
    classes=len(source_subset),
    subset_classes=source_subset,
)
source_test = torch.utils.data.TensorDataset(
    torch.stack(source_subset_samples), torch.LongTensor(source_labels)
)


target_test = target(
    root=DATASET_PATH, train=False, download=True, transform=target_transform
)
target_subset_samples, target_labels, target_class_indicies = get_indicies_subset(
    target_test,
    new_labels=new_labels_target,
    classes=len(target_subset),
    subset_classes=target_subset,
)
target_test = torch.utils.data.TensorDataset(
    torch.stack(target_subset_samples), torch.LongTensor(target_labels)
)

full_set_test = SubsetGuidedDataset(
    source_test,
    target_test,
    num_labeled="all",
    in_indicies=source_class_indicies,
    out_indicies=target_class_indicies,
)

XY_test_sampler = SubsetGuidedSampler(full_set_test, subsetsize=1)

# for accuracy
X_test_loader = torch.utils.data.DataLoader(
    source_test,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
)
Y_test_loader = torch.utils.data.DataLoader(
    target_test,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
)

## 4. Testing


### init models


In [ ]:
T = CUNet(
    DATASET1_CHANNELS, DATASET2_CHANNELS, TIME_DIM, base_factor=UNET_BASE_FACTOR
).cuda()

T = SDE(
    shift_model=T,
    epsilon=EPSILON,
    n_steps=N_STEPS,
    time_dim=TIME_DIM,
    n_last_steps_without_noise=N_LAST_STEPS_WITHOUT_NOISE,
    use_positional_encoding=USE_POSITIONAL_ENCODING,
    use_gradient_checkpoint=USE_GRADIENT_CHECKPOINT,
    predict_shift=PREDICT_SHIFT,
    image_input=IMAGE_INPUT,
).cuda()

print("T params:", np.sum([np.prod(p.shape) for p in T.parameters()]))

### Load weights for continue training

In [ ]:
print("Loading weights")

w_path = os.path.join(LOAD_PATH, f"T_{SEED}_5000.pt")  # user setting

T.load_state_dict(torch.load(w_path))

print(f"{w_path}, loaded")

### Plots Test


In [ ]:
X_test_fixed, Y_test_fixed = XY_test_sampler.sample(10)
X_test_fixed, Y_test_fixed = X_test_fixed.flatten(0, 1), Y_test_fixed.flatten(0, 1)

In [ ]:
fig, axes = plot_sde_pushed_images(X_test_fixed, Y_test_fixed, T, gray=GRAY_PLOTS)

In [ ]:
fig, axes = plot_sde_pushed_random_class_images(XY_test_sampler, T, gray=GRAY_PLOTS)

### main testing


In [ ]:
clear_output(wait=True)

print("Plotting")

inference_T = T
inference_T.eval()
print("Fixed Test Images")
fig, axes = plot_sde_pushed_images(
    X_test_fixed, Y_test_fixed, inference_T, gray=GRAY_PLOTS, plot_trajectory=False
)
plt.show(fig)
plt.close(fig)

print("Random Test Images")
fig, axes = plot_sde_pushed_random_class_images(
    XY_test_sampler, inference_T, gray=GRAY_PLOTS, plot_trajectory=False
)
plt.show(fig)
plt.close(fig)

In [ ]:
print("Fixed Test Images Trajectory")
fig, axes = plot_sde_pushed_images(
    X_test_fixed, Y_test_fixed, inference_T, gray=GRAY_PLOTS
)
plt.show(fig)
plt.close(fig)

print("Random Test Images Trajectory")
fig, axes = plot_sde_pushed_random_class_images(
    XY_test_sampler, inference_T, gray=GRAY_PLOTS
)
plt.show(fig)
plt.close(fig)

In [ ]:
print("Computing FID")
target_mu, target_sigma = get_loader_stats(
    Y_test_loader, BATCH_SIZE, FID_EPOCHS, verbose=True, use_Y=False
)
gen_mu, gen_sigma = get_sde_pushed_loader_stats(
    inference_T,
    X_test_loader,
    n_epochs=FID_EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=True,
)
fid = calculate_frechet_distance(gen_mu, gen_sigma, target_mu, target_sigma)
print(f"FID={fid}")

In [ ]:
print("Computing Accuracy")
accuracy = get_sde_pushed_loader_accuracy(inference_T, X_test_loader, classifier)

In [ ]:
print("Computing Metrics")
metrics = get_sde_pushed_loader_metrics(
    T,
    XY_test_sampler.loader,
    n_epochs=FID_EPOCHS,
    verbose=True,
    log_metrics=["LPIPS", "PSNR", "SSIM", "MSE", "MAE"],
)
print(f"metrics={metrics}")